In [1]:
import requests, time, sys
from bs4 import BeautifulSoup

In [2]:
# Function to attempt to retrieve
def get_attempt(base_url, get_params=None, attempts=10):
    for attempt in range(attempts):
        time.sleep(0.5)
        try:
            response = requests.get(base_url, params=get_params, allow_redirects=True)
            break
        except:
            if attempt == attempts - 1:
                response = "FAILED"
            continue
    if response == "FAILED":
        print(f"Could not search {base_url}")
    return response

In [3]:
def check_page(href, done_hrefs, targets, hit_hrefs):
    done_hrefs.append(href)
    req = get_attempt(done_hrefs[0] + href)
    if req == "FAILED":
        return
    html = req.content
    soup = BeautifulSoup(html, 'html.parser')
    anchors = soup.find_all("a")
    for item in anchors:
        if 'href' in item.attrs.keys():
            if item['href'] in targets: 
                print("Hit!", href, item)
                hit_hrefs.append([href, item])
            elif (item['href'] not in done_hrefs) and (item['href'] not in done_hrefs) and (item['href'][:4] != "http") and (item['href'].find('?') == -1):
                check_page(item['href'], done_hrefs, targets, hit_hrefs)
    return

In [4]:
root_url = input("Please enter a root url: ")
done_hrefs = [root_url, '/']
hit_hrefs = []

targets = []
while True:
    new_target = input("Please enter a target url or press enter: ")
    if new_target != "":
        targets.append(new_target)
        if targets[-1][-1] == "/":
            targets.append(new_target[:-1])
        else:
            targets.append(new_target + "/")
    else:
        break

r = get_attempt(root_url)
if r == "FAILED":
    print("Failed to access root url")
    sys.exit()
root_html = r.content
root_soup = BeautifulSoup(root_html, 'html.parser')
home_links = root_soup.find_all('a')

home_hrefs = []
for link in home_links:
    if ('href' in link.attrs.keys()) and (link['href'] not in home_hrefs) and (link['href'].find('?') == -1) and (link['href'][:4] != "http"):
        home_hrefs.append(link['href'])

for home_href in home_hrefs:
    print(f"Now processing {home_hrefs.index(home_href)+1} of {len(home_hrefs)}, {home_href}")
    check_page(home_href, done_hrefs, targets, hit_hrefs)
        
print(hit_hrefs)

Please enter a root url:  https://property.dcra.dc.gov/#!/landingPage
Please enter a target url or press enter:  https://acaprod9.dcra.dc.gov/CitizenAccess/Default.aspx
Please enter a target url or press enter:  https://acaprod2.dcra.dc.gov/dcrapublicprod1
Please enter a target url or press enter:  


[]


In [5]:
print(len(hit_hrefs))

0


In [6]:
for hit in hit_hrefs:
    print(f"{root_url + hit[0]}, {hit[1]}")